$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\Xvh}{\hat{\mathbf{X}}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\Yv}{\mathbf{Y}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\gv}{\mathbf{g}}
\newcommand{\Hv}{\mathbf{H}}
\newcommand{\dv}{\mathbf{d}}
\newcommand{\Vv}{\mathbf{V}}
\newcommand{\vv}{\mathbf{v}}
\newcommand{\Uv}{\mathbf{U}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\Zv}{\mathbf{Z}}
\newcommand{\Zvh}{\hat{\mathbf{Z}}}
\newcommand{\Ev}{\mathbf{E}}
\newcommand{\onev}{\mathbf{1}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}
$


# A2.1 Three-Layer Neural Network

For this assignment, we modify the two-layer neural network matrix equations and code from lecture notes to become a three-layer neural network, one with two hidden layers.  In the following diagram, you will be adding another layer on the left between in the inputs, $x$, and the show hiddden layer.

You must complete three steps.
1. Complete the matrix equations using latex notation in the markdown cell,
2. Implement the required functions in python.
3. Perform the described experiments.

## Network Diagram
Three-layer neural network with two hidden layers

![Two Layers](twoHiddenLayers.png)

$$ 
\begin{align*}
N &= \text{ number of samples } \\
I &= \text{ the number of attributes in each sample }\\
K &= \text{ number of units in output layer } \\
H_1 &= \text{ number of units in first hidden layer }\\
H_2 &= \text{ number of units in second hidden layer }\\
~\\
\Zv_1 &= \tanh(\Xvh\, \Uv) \\
\Zv_2 &= \tanh(\Zvh_1\, \Vv) \\
\Yv &= \Zvh_2\, \Wv\\
\Ev &= \frac{1}{NK} \sum_{n=1}^N \sum_{k=1}^K (\Tv_{n,k} - \Yv_{n,k})^2 \\
~\\
%------------------------------------
% Gradient of 'E' with respect to 'Y'
\nabla_\Yv E_{n,k} &= \frac{-2}{NK} (\Tv_{n,k} - \Yv_{n,k})\\
~\\ 
%------------------------------------
% Gradient of 'E' with respect to 'W'
\nabla_\Wv E &=  \underbrace{\underbrace{\Zvh_2^T}_{H+1\times N} \underbrace{\delta_\Yv}_{N\times K}}_{H+1\times K} \;\;\;\;\text{ where } \delta_\Yv = \frac{-2}{NK} (\Tv - \Yv)\\
~\\ 
%------------------------------------
% Gradient of 'E' with respect to 'V'
\nabla_\Vv E &= \underbrace{\underbrace{\Zvh_1^T}_{I+1\times N} \; \underbrace{\delta_{\Zv_2}}_{N\times H}}_{I+1\times H} \;\;\;\;\text{ where } \delta_{\Zv_2} = (\delta_\Yv \; \Wv_{1:}^T)\; \cdot\; (1-\Zv_2^2) \;\;\;\; \text{ if } f(\Zvh_1 \Vv) = \tanh(\Xvh \Vv)\\
~\\ 
%------------------------------------
% Gradient of 'E' with respect to 'U'
\nabla_\Uv E &= \underbrace{\underbrace{\Xvh^T}_{I+1\times N} \; \underbrace{\delta_{\Zv_1}}_{N\times H}}_{I+1\times H} \;\;\;\;\text{ where } \delta_{\Zv_2} = (\delta_\Yv \; \Wv_{1:}^T)\; \cdot\; (1-\Zv_1^2) \;\;\;\; \text{ if } f(\Xvh \Vv) = \tanh(\Xvh \Vv)\\
\end{align*}
$$



## Code

Modify the functions `network` `error_gradient` and `mse` to do the compuations for a three-layer neural network.  They must have the following arguments:
* `Y = network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, all_outputs=False)`
or
* `Y, Z1, Z2 = network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, all_outputs=True)`
* `gradient = error_gradient(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T)`
* `mean_squared_error = mse(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T)`

In [1]:
### From Lecture 06 - Make code cell if needed
def network__06(w, n_inputs, n_hiddens, n_outputs, X, all_outputs=False):
    n_V = (n_inputs + 1) * n_hiddens
    n_W = (n_hiddens + 1) * n_outputs
    V = w[:n_V].reshape((n_inputs + 1, n_hiddens))
    W = w[n_V:].reshape((n_hiddens + 1, n_outputs))
    Z = np.tanh(V[0:1, :] + X @ V[1:, :])
    Y = W[0:1, :] + Z @ W[1:, :]
    return (Y, Z) if all_outputs else Y

def error_gradient__06(w, n_inputs, n_hiddens, n_outputs, X, T):
    Y, Z = network(w, n_inputs, n_hiddens, n_outputs, X, all_outputs=True)
    n_samples = X.shape[0]
    delta_Y = -2 / n_samples * (T - Y)
    Z_hat = np.insert(Z, 0, 1, axis=1)
    dEdW = Z_hat.T @ delta_Y
    
    n_W = (n_hiddens + 1) * n_outputs
    W = w[-n_W:].reshape((n_hiddens + 1, n_outputs))
    delta_Z = (delta_Y @ W[1:, :].T) * (1 - Z**2)
    X_hat = np.insert(X, 0, 1, axis=1)
    dEdV = X_hat.T @ delta_Z
 
    dEdw = np.hstack((dEdV.flatten(), dEdW.flatten()))

    return dEdw

def mse__06(w, n_inputs, n_hiddens, n_outputs, X, T):
    Y = network(w, n_inputs, n_hiddens, n_outputs, X)
    return np.mean((T - Y)**2)

In [2]:
#def network__06(w, n_inputs, n_hiddens, n_outputs, X, all_outputs=False):
#    n_V = (n_inputs + 1) * n_hiddens
#    n_W = (n_hiddens + 1) * n_outputs
#    V = w[:n_V].reshape((n_inputs + 1, n_hiddens))
#    W = w[n_V:].reshape((n_hiddens + 1, n_outputs))
#    Z = np.tanh(V[0:1, :] + X @ V[1:, :])
#    Y = W[0:1, :] + Z @ W[1:, :]
#    return (Y, Z) if all_outputs else Y
def network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, all_outputs=False):
    n_U = (n_inputs + 1) * n_hiddens_1
    n_V = (n_hiddens_1 + 1) * n_hiddens_2
    n_W = (n_hiddens_2 + 1) * n_outputs
    U = w[0:n_U].reshape((n_inputs + 1, n_hiddens_1))
    V = w[n_U: n_U + n_V].reshape((n_hiddens_1 + 1, n_hiddens_2))
    W = w[n_U + n_V:].reshape((n_hiddens_2 + 1, n_outputs))

#   Z   = np.tanh(V[0:1, :] + X @ V[1:, :])    
    Z_1 = np.tanh(U[0:1, :] + X @ U[1:, :])
    Z_2 = np.tanh(V[0:1, :] + Z_1 @ V[1:, :])
    Y = W[0:1, :] + Z_2 @ W[1:, :]
    return (Y, Z) if all_outputs else Y 

### Verification from A2grader.py

In [4]:
import numpy as np
import matplotlib.pyplot as plt

import pandas
import optimizers as opt  # from Lecture Notes

In [5]:
X = np.arange(3 * 4).reshape((3, 4)) * 0.1
T = np.hstack(( np.sin(X[:, 0:1]) + X[:, 1:2],
                X[:, 2:3] * -0.5,
                X[:, 3:4] ** 2))
n_inputs = X.shape[1]
n_outputs = T.shape[1]
n_hiddens_1 = 6
n_hiddens_2 = 2
n_w = (n_inputs + 1) * n_hiddens_1 + (n_hiddens_1 + 1) * n_hiddens_2 + (n_hiddens_2 + 1) * n_outputs
w = (np.arange(n_w) - n_w/2) * 0.01
# Y = network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X)

In [6]:
Y = network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X)

In [7]:
Y

array([[0.12022254, 0.12773632, 0.13525011],
       [0.08581156, 0.09176614, 0.09772072],
       [0.05582873, 0.06042448, 0.06502022]])

In [ ]:
#def error_gradient__06(w, n_inputs, n_hiddens, n_outputs, X, T):
#    Y, Z = network(w, n_inputs, n_hiddens, n_outputs, X, all_outputs=True)
#    n_samples = X.shape[0]
#    delta_Y = -2 / n_samples * (T - Y)
#    Z_hat = np.insert(Z, 0, 1, axis=1)
#    dEdW = Z_hat.T @ delta_Y
#    
#    n_W = (n_hiddens + 1) * n_outputs
#    W = w[-n_W:].reshape((n_hiddens + 1, n_outputs))
#    delta_Z = (delta_Y @ W[1:, :].T) * (1 - Z**2)
#    X_hat = np.insert(X, 0, 1, axis=1)
#    dEdV = X_hat.T @ delta_Z
# 
#    dEdw = np.hstack((dEdV.flatten(), dEdW.flatten()))
#
#    return dEdw
def error_gradient(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T):
    Y, Z = network(w, n_inputs, n_hiddens, n_outputs, X, all_outputs=True)
    n_samples = X.shape[0]
    delta_Y = -2 / n_samples * (T - Y)
    Z_hat = np.insert(Z, 0, 1, axis=1)
    dEdW = Z_hat.T @ delta_Y
    
    n_W = (n_hiddens + 1) * n_outputs
    W = w[-n_W:].reshape((n_hiddens + 1, n_outputs))
    delta_Z = (delta_Y @ W[1:, :].T) * (1 - Z**2)
    X_hat = np.insert(X, 0, 1, axis=1)
    dEdV = X_hat.T @ delta_Z
 
    dEdw = np.hstack((dEdV.flatten(), dEdW.flatten()))

    return dEdw

In [ ]:
#def mse__06(w, n_inputs, n_hiddens, n_outputs, X, T):
#    Y = network(w, n_inputs, n_hiddens, n_outputs, X)
#    return np.mean((T - Y)**2)
# THIS SHOULD BE GOOD TO GO
def mse(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T):
    Y =  network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X)
    return np.mean((T - Y)**2)

## Test on Air Quality Data
Use your code to, as before, predict CO from the Hour of the day.

Set up the data matrices `X` and `T` as follows.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pandas
import optimizers as opt  # from Lecture Notes

In [ ]:
data = pandas.read_csv('AirQualityUCI.csv', delimiter=';', decimal=',', usecols=range(15), na_values=-200)
data = data[['Time', 'CO(GT)']]
data = data [:23 * 20]  # first 20 days of data
data = data.dropna(axis=0)
print('data.shape =', data.shape)

hour = [int(t[:2]) for t in data['Time']]
X = np.array(hour).reshape(-1, 1)
CO = data['CO(GT)']
T = np.array(CO).reshape(-1, 1)
np.hstack((X, T))[:10]  # show the first 10 samples of hour, CO

Define the network parameters.  Use the shapes of `X` and `T` to assign the number of inputs and outputs.  Define each of the two hidden layers to have 5 units.

In [ ]:
n_inputs = X.shape[1]
n_hiddens_1 = 5
n_hiddens_2 = 5
n_outputs = T.shape[1]

Now we must define the intial weight vector.  The vector contains a value for each weight in all three layers.

In [ ]:
n_U = (n_inputs + 1) * n_hiddens_1
n_V = (n_hiddens_1 + 1) * n_hiddens_2
n_W = (n_hiddens_2 + 1) * n_outputs

initial_w = np.random.uniform(-0.1, 0.1, n_U + n_V + n_W)  # range of weights is -0.1 to 0.1

Before training our network, standardize the input values, to change the hour to have zero mean and unit variance across the set of samples.  Change `True` to `False` to not perform this step, allowing you to compare results with and without standardization.  It is not required to show the results here.  When you check in your notebook, leave `standardize` set to `True`.

In [ ]:
standardize = True

if standardize:
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    
    X = (X - X_mean) / X_std
    
print(f'X mean is {X.mean(axis=0)[0]:.3f} and its standard deviation is {X.std(axis=0)[0]:.3f}')

Now we can train our network using each of our three optimization algorithms.

In [ ]:
n_iterations = 2000

result_sgd = opt.sgd(initial_w,
                     mse, error_gradient, fargs=[n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T],
                     n_iterations=n_iterations, learning_rate=1e-1, momentum_rate=0.2, 
                     save_wtrace=True)
#print(f'SGD final error is {result_sgd["ftrace"][-1]:.3f} and it took {result_sgd["time"]:.2f} seconds')

#result_adam = opt.adam(initial_w, 
#                       mse, error_gradient, fargs=[n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T],
#                       n_iterations=n_iterations, learning_rate=1e-2, 
#                       save_wtrace=True)
#print(f'Adam final error is {result_adam["ftrace"][-1]:.3f} and it took {result_adam["time"]:.2f} seconds')

#result_scg = opt.scg(initial_w,
#                     mse, error_gradient, fargs=[n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T],
#                     n_iterations=n_iterations,
#                     save_wtrace=True)
#print(f'SCG final error is {result_scg["ftrace"][-1]:.3f} and it took {result_scg["time"]:.2f} seconds')

Examine the error curve and the model fits for each of the optimization algorithms.

In [ ]:
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(result_sgd['ftrace'], label='SGD')
plt.plot(result_adam['ftrace'], label='Adam')
plt.plot(result_scg['ftrace'], label='SCG')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.ylim(0, 4)

plt.subplot(1, 2, 2)
if standardize:
    plt.plot(X * X_std + X_mean, T, 'k.')  # unstandardize X
else:
    plt.plot(X, T, 'k.')
xs = np.linspace(0, 23, 100).reshape((-1, 1))
xs_standardized = (xs - X_mean) / X_std if standardize else xs
plt.plot(xs, network(result_sgd['w'], n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, xs_standardized), label='SGD')
plt.plot(xs, network(result_adam['w'], n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, xs_standardized), label='Adam')
plt.plot(xs, network(result_scg['w'], n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, xs_standardized), label='SCG')
plt.legend()
plt.xlabel('Hour')
plt.ylabel('CO');

Here is some code that tries a number of values for the key parameters of `n_iterations`, `n_hiddens_1`, `n_hiddens_2` and `learning_rate`.

**Required:** Modify the lists of values in the four for loops to try other parameter values.  Try to find ranges that work well for all three algorithms.

In [ ]:
results = []
for n_iterations in [10, 100]:
    for nh1 in [1, 2]:
        for nh2 in [1, 2]:
            
            n_U = (n_inputs + 1) * nh1
            n_V = (nh1 + 1) * nh2
            n_W = (nh2 + 1) * n_outputs
            initial_w = np.random.uniform(-0.1, 0.1, n_U + n_V + n_W)

            result_scg = opt.scg(initial_w, mse, error_gradient, fargs=[n_inputs, nh1, nh2, n_outputs, X, T],
                                 n_iterations=n_iterations)
            results.append([n_iterations, nh1, nh2, 0, 'scg', result_scg['ftrace'][-1]])
            
            for lr in [1e-3, 1e-5]:
                
                result_sgd = opt.sgd(initial_w, mse, error_gradient, fargs=[n_inputs, nh1, nh2, n_outputs, X, T],
                     n_iterations=n_iterations, learning_rate=lr, momentum_rate=0)
                result_adam = opt.adam(initial_w, mse, error_gradient, fargs=[n_inputs, nh1, nh2, n_outputs, X, T],
                                       n_iterations=n_iterations, learning_rate=lr)
                results.append([n_iterations, nh1, nh2, 'sgd', result_scg['ftrace'][-1]])

                results.append([n_iterations, nh1, nh2, lr, 'sgd', result_sgd['ftrace'][-1]])
                results.append([n_iterations, nh1, nh2, lr, 'adam', result_adam['ftrace'][-1]])

results = pandas.DataFrame(results, columns=('Iterations', 'nh1', 'nh2', 'lr', 'algo', 'mse'))
results

**Required:** Show the results for the 20 lowest MSE values, sorted by increasing MSE. Read about the `sort_values` and `head` methods on a `DataFrame`.  You can do this with a single line of python code.

In [ ]:
# ...code here....  Can be just one line of python

Since you have cleverly written your code to handle any value of $I$, $H_1$, $H_2$, and $K$, your code should be able to handle the following data that contains three attributes for each of 5 samples and two output values for each sample.

In [ ]:
X = np.arange(15).reshape((5, 3))
T = np.hstack((X[:, 0:1] * 0.1 * X[:, 1:2], X[:, 2:]**2)) # making two target values for each sample
T = T.reshape((5, 2))
print('  Input            Target')
for x, t in zip(X, T):
    print(x, '\t', t)

Let's use your code to train a neural network with two hidden layers, having 50 units in the first hidden layer and 3 units in the second hidden layer.  This example does not perform the standardization step.

In [ ]:
n_hiddens_1 = 50
n_hiddens_2 = 3
n_iterations = 1000


n_inputs = X.shape[1]
n_outputs = T.shape[1]

n_U = (n_inputs + 1) * n_hiddens_1
n_V = (n_hiddens_1 + 1) * n_hiddens_2
n_W = (n_hiddens_2 + 1) * n_outputs

initial_w = np.random.uniform(-0.1, 0.1, n_U + n_V + n_W)  # range of weights is -0.1 to 0.1

result_sgd = opt.sgd(initial_w,
                     mse, error_gradient, fargs=[n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T],
                     n_iterations=n_iterations, learning_rate=1e-1, momentum_rate=0.2, 
                     save_wtrace=True)
print(f'SGD final error is {result_sgd["ftrace"][-1]:.3f} and it took {result_sgd["time"]:.2f} seconds')

result_adam = opt.adam(initial_w, 
                       mse, error_gradient, fargs=[n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T],
                       n_iterations=n_iterations, learning_rate=1e-2, 
                       save_wtrace=True)
print(f'Adam final error is {result_adam["ftrace"][-1]:.3f} and it took {result_adam["time"]:.2f} seconds')

result_scg = opt.scg(initial_w,
                     mse, error_gradient, fargs=[n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T],
                     n_iterations=n_iterations,
                     save_wtrace=True)
print(f'SCG final error is {result_scg["ftrace"][-1]:.3f} and it took {result_scg["time"]:.2f} seconds')

In [ ]:
w = result_scg['w']

Y = network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X)
Y.shape

In [ ]:
def plot_diagonal(T, Y):
    a = min(T.min(), Y.min())
    b = max(T.max(), Y.max())
    plt.plot([a, b], [a, b], '-', lw=3, alpha=0.5)

plt.figure(figsize=(12, 5))

plt.subplot(1, 3, 1)
plt.plot(result_scg['ftrace'])
plt.xlabel('Iteration')
plt.ylabel('MSE')
plt.title('SCG')

plt.subplot(1, 3, 2)
plt.plot(T[:, 0], Y[:, 0], '.')
plot_diagonal(T[:, 0], Y[:, 0])
plt.xlabel('Target')
plt.ylabel('Prediction')
plt.title('$Y_1$')

plt.subplot(1, 3, 3)
plt.plot(T[:, 1], Y[:, 1], '.')
plot_diagonal(T[:, 1], Y[: 1])
plt.xlabel('Target')
plt.ylabel('Prediction')
plt.title('$Y_2$');

## Grading

Your notebook will be run and graded automatically. Test this grading process by first downloading [A2grader.zip](http://www.cs.colostate.edu/~anderson/cs545/notebooks/A2grader.zip) and extract `A2grader.py` from it. Run the code in the following cell to demonstrate an example grading session. You should see a perfect execution score of 80/80 if your functions are defined correctly. The remaining 20 points will be based on other testing and the results you obtain and your discussions. 

A different, but similar, grading script will be used to grade your checked-in notebook. It will include additional tests. You should design and perform additional tests on all of your functions to be sure they run correctly before checking in your notebook.  

For the grading script to run correctly, you must first name this notebook as 'Lastname-A2.ipynb' with 'Lastname' being your last name, and then save this notebook.

In [8]:
%run -i A2grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Staab-A2.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.

Testing
X = np.arange(3 * 4).reshape((3, 4)) * 0.1
T = np.hstack(( np.sin(X[:, 0:1]) + X[:, 1:2],
                X[:, 2:3] * -0.5,
                X[:, 3:4] ** 2))
n_inputs = X.shape[1]
n_outputs = T.shape[1]
n_hiddens_1 = 6
n_hiddens_2 = 2
n_w = (n_inputs + 1) * n_hiddens_1 + (n_hiddens_1 + 1) * n_hiddens_2 + (n_hiddens_2 + 1) * n_outputs
w = (np.arange(n_w) - n_w/2) * 0.01
Y = network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X)


--- 20/20 points. Returned correct values.

Testing
Y, Z1, Z2 = network(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, all_outputs=True)


--- 0/20 points. network raised the exception

name 'Z' is not defined

Testing
grad = error_gradient(w, n_inputs, n_hiddens_1, n_hiddens_2, n_outputs, X, T)


--- 0/20 p

## Check-In

Do not include this section in your notebook.

Name your notebook ```Lastname-A2.ipynb```.  Submit the file using the ```Assignment 2``` link on [Canvas](https://colostate.instructure.com/courses/86986).

## Extra Credit

Train a neural network with two hidden layers with five units in each layer to predict CO from the Hour of the day.  After it is trained plot the outputs of the five units in the first hidden layer versus 100 floating point values of hour from 0 to 23 in one subplot of one figure.  Then, in another subplot plot the outputs of the five units in the second hidden layer.  In a third subplot, plot the predicted CO from the output of the neural network.  Arrange the subplots vertically.  Describe what you see in the plots.